In [2]:
!pip install transformers[torch] -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 115.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 18.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 18.0.0 which is incompatible.


In [4]:
pip install Bio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.0/281.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 73.5 MB/s eta 0:00:00


In [5]:

!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.8 MB/s eta 0:00:00


In [6]:
import os
from Bio import SeqIO
import random

fasta_dir = "/content/drive/MyDrive/s1116049/EFFLUX_NEW_WORK/fad.sites"

# Get a list of all FASTA files in the directory
fasta_files = [f for f in os.listdir(fasta_dir) if f.endswith(".fasta")]

# Shuffle the list of files
random.shuffle(fasta_files)

# Determine the number of files for training and validation
train_ratio = 0.8  # 80% for training
num_files = len(fasta_files)
num_train = int(train_ratio * num_files)

# Split the files into training and validation sets
train_files = fasta_files[:num_train]
val_files = fasta_files[num_train:]

# Save the sequences of the training set to a text file
train_sequences_path = "/content/drive/MyDrive/s1116049/EFFLUX_NEW_WORK/fad.sites/FAD_train_sequences.txt"
with open(train_sequences_path, "w") as train_file:
    for file in train_files:
        file_path = os.path.join(fasta_dir, file)
        try:
            for record in SeqIO.parse(file_path, "fasta"):
                train_file.write(f"{record.seq}\n\n")
        except FileNotFoundError:
            print(f"File not found: {file_path}")

print("Sequences of the training set saved to:", train_sequences_path)

# Create the validation directory if it doesn't exist
validation_dir = "/content/drive/MyDrive/s1116049/EFFLUX_NEW_WORK/fad.sites/validation/"
os.makedirs(validation_dir, exist_ok=True)

# Save the sequences of the validation set to a text file
val_sequences_path = os.path.join(validation_dir, "FAD_validation_sequences.txt")
with open(val_sequences_path, "w") as val_file:
    for file in val_files:
        file_path = os.path.join(fasta_dir, file)
        try:
            for record in SeqIO.parse(file_path, "fasta"):
                val_file.write(f"{record.seq}\n")
        except FileNotFoundError:
            print(f"File not found: {file_path}")

print("Sequences of the validation set saved to:", val_sequences_path)


Sequences of the training set saved to: /content/drive/MyDrive/s1116049/EFFLUX_NEW_WORK/fad.sites/FAD_train_sequences.txt
Sequences of the validation set saved to: /content/drive/MyDrive/s1116049/EFFLUX_NEW_WORK/fad.sites/validation/FAD_validation_sequences.txt


In [7]:
import os
import random
import torch
from datasets import Dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling
from torch.utils.data import DataLoader

# Paths for training and validation sequences
train_file_path = "/content/drive/MyDrive/s1116049/EFFLUX_NEW_WORK/fad.sites/FAD_train_sequences.txt"
valid_file_path = "/content/drive/MyDrive/s1116049/EFFLUX_NEW_WORK/fad.sites/validation/FAD_validation_sequences.txt"

# Read the sequences from files
with open(train_file_path, "r") as train_file:
    train_sequences = train_file.read().splitlines()

with open(valid_file_path, "r") as valid_file:
    val_sequences = valid_file.read().splitlines()

# Create datasets from sequences
train_dataset = Dataset.from_dict({"text": train_sequences})
valid_dataset = Dataset.from_dict({"text": val_sequences})

# Model and tokenizer parameters
model_name = "nferruz/ProtGPT2"
output_dir = "/content/drive/MyDrive/s1116049/EFFLUX_NEW_WORK/fad.sites/Model_FAD"
learning_rate = 1e-6
max_length = 1467  # Adjust to your needs

# Load pretrained model and tokenizer
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Set device to CPU for debugging or GPU for actual training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Tokenization function with padding and truncation
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=max_length)

# Apply tokenization to datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)

# Ensure dataset is in the correct format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
valid_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

# Data collator for dynamic padding during training
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# DataLoader with small batch size for debugging
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=data_collator)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, collate_fn=data_collator)

# Set environment variable to capture CUDA errors for debugging
%env CUDA_LAUNCH_BLOCKING=1

# Example training step for debugging (one batch)
try:
    for batch in train_loader:
        # Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}
        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss
        print("Training Loss:", loss.item())
        break  # Remove this break for actual training
except RuntimeError as e:
    print(f"Runtime error: {e}")

# Once verified, increase batch size and run training loop (after debugging)
# Remove `CUDA_LAUNCH_BLOCKING=1` for regular GPU training
# train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=data_collator)
# valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False, collate_fn=data_collator)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/850 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/655k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/314k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

env: CUDA_LAUNCH_BLOCKING=1
Runtime error: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.



In [8]:
from transformers import Trainer, TrainingArguments
from torch.cuda.amp import autocast

# Define the training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    learning_rate=learning_rate,
    num_train_epochs=10,
    per_device_train_batch_size=2,  # Reduce batch size
    per_device_eval_batch_size=2,    # Reduce batch size
    save_steps=200,
    save_total_limit=5,
    evaluation_strategy="steps",
    logging_steps=100,
    eval_steps=200,
    dataloader_num_workers=4,
    gradient_accumulation_steps=8,  # Increase gradient accumulation steps
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset
)

# Start training with mixed precision
with trainer.accelerator.scaler.scale(scale=1.0 / trainer.gradient_accumulation_steps):
    trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
import os
from Bio import SeqIO

def find_max_sequence_length(folder_path):
    max_length = 0
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".fasta") or file_name.endswith(".fa"):
            file_path = os.path.join(folder_path, file_name)
            for record in SeqIO.parse(file_path, "fasta"):
                sequence_length = len(record.seq)
                if sequence_length > max_length:
                    max_length = sequence_length
    return max_length

# Example usage
folder_path = '/content/drive/MyDrive/s1116049/DNA_RNA_binding_Site/DNA_binding_fasta/TE46'  # Replace with your folder path
max_sequence_length = find_max_sequence_length(folder_path)
print("Maximum sequence length:", max_sequence_length)
